In [1]:
from open3d import *
import numpy as np 
import csv
import os


In [2]:
def read_dat(file_path):

    content = []
    ptcloud = []

    file = open(file_path, 'r')
    for line in file:
        fields = line.split(' ')
        
        robot_pose_x = float(fields[0])
        robot_pose_y = float(fields[1])
        robot_pose_z = float(fields[2])
        point_x = float(fields[3])
        point_y = float(fields[4])
        point_z = float(fields[5])
        
        content_all = np.array([robot_pose_x, robot_pose_y, robot_pose_z, point_x, point_y, point_z])
        content_ptcloud = np.array([point_x, point_y, point_z])
        
        content.append(content_all)
        ptcloud.append(content_ptcloud)

    content = np.array(content)
    ptcloud = np.array(ptcloud)
    
    return content, ptcloud
    

In [44]:
"""
Read Points (from dat file)
"""

file_dir = '/home/irap/Documents/data/freiburgCampus360_3D/freiburgCampus360_3D/'
scan_num = '001'

file_path_local = 'scan_' + scan_num + '_points.dat'
file_path_local_nameonly = 'scan_' + scan_num + '_points'
file_path = file_dir + 'scan_' + scan_num + '_points.dat'

content_all, ptcloud = read_dat(file_path)

In [4]:
ptcloud

array([[-10.276,   5.891,   0.646],
       [-10.677,   5.928,   0.505],
       [-10.181,   5.474,   0.384],
       ...,
       [ -4.028,  -2.345,   0.842],
       [ -9.152,  -5.192,   0.876],
       [ -3.269,  -2.07 ,   0.729]])

In [64]:
"""
Convert numpy array into ply form (available for open3d functions)
"""

ptcloud_file = PointCloud()
ptcloud_file.points = Vector3dVector(ptcloud)

save_file_path = './data/' + file_path_local_nameonly + '.ply' 
write_point_cloud(save_file_path, ptcloud_file)


True

In [65]:
# Load saved point cloud and visualize it
pcd_load = read_point_cloud(save_file_path)
draw_geometries([pcd_load])


In [13]:
"""
Downsampling
"""
print("Downsample the point cloud with a voxel of ")
downpcd = voxel_down_sample(pcd_load, voxel_size = 0.05)
draw_geometries([downpcd])




Downsample the point cloud with a voxel of 


In [14]:
print(pcd_load)
print(downpcd)

PointCloud with 176250 points.
PointCloud with 103544 points.


In [41]:
"""
removing road
"""
# calc normals
estimate_normals(pcd_load, search_param = KDTreeSearchParamHybrid(
        radius = 1, max_nn = 30)) # empirically, radius 1 was good.
# draw_geometries([pcd_load])


# numpy points for processing
points = np.asarray(pcd_load.points)
num_points = len(points)
print(num_points)

176250


In [38]:
# removing road points 
normals = np.asarray(pcd_load.normals)
road_removed_points = []
for i in range(num_points):
    ith_point_xyz = points[i, :]
    ith_point_normal = normals[i, :]

    z_normal = ith_point_normal[2]
    e = 0.05
    if( np.abs( np.abs(z_normal) - 1) < e):
        pass
    else:
        road_removed_points.append(ith_point_xyz)
        
road_removed_points = np.array(road_removed_points)

In [48]:
# save
ptcloud_file = PointCloud()
ptcloud_file.points = Vector3dVector(road_removed_points)

road_removed_file_path_local_nameonly = file_path_local_nameonly + '_without_road'
save_file_path = './data/' + road_removed_file_path_local_nameonly + '.ply' 
write_point_cloud(save_file_path, ptcloud_file)

# Load saved point cloud and visualize it
pcd_without_road_load = read_point_cloud(save_file_path)
draw_geometries([pcd_without_road_load])


In [62]:
"""
Downsampling of the road-removed point cloud
"""
downpcd_without_road_load = voxel_down_sample(pcd_without_road_load, voxel_size = 0.3)
draw_geometries([downpcd_without_road_load])

# save 
downsampled_road_removed_file_path_local_nameonly = file_path_local_nameonly + '_without_road_downsampled'
save_file_path = './data/' + downsampled_road_removed_file_path_local_nameonly + '.ply' 
write_point_cloud(save_file_path, downpcd_without_road_load)

# number of downsampled points (without road)
print(downpcd_without_road_load)

PointCloud with 11749 points.


PointCloud with 11749 points.

In [57]:
pcd_without_road_load

PointCloud with 100080 points.

In [35]:
points[4,:]

array([-8.932,  4.496,  0.199])

In [40]:
road_removed_points.shape

(100080, 3)

In [37]:
np.abs(-2)

2